In [ ]:
# 1. 라이브러리 설치 (코랩에서 한 번만)
!pip install imbalanced-learn scikit-learn pandas

# 2. 라이브러리 임포트
import pandas as pd
from imblearn.over_sampling import SMOTE
from google.colab import files

# 3. CSV 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 4. 데이터 불러오기 (인코딩 지정)
try:
    df = pd.read_csv(filename, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(filename, encoding='cp949')  # 한국어 Windows CSV 대부분
print("데이터 불러오기 완료!")

# 5. 데이터 구조 확인
print("데이터 예시:")
print(df.head())
print("\n라벨 분포:")
print(df['label'].value_counts())

# 6. SMOTE 적용
X = df.drop('label', axis=1)  # 특성
y = df['label']               # 라벨

smote = SMOTE(random_state=42, k_neighbors=2)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 7. SMOTE 결과 확인
print("\nSMOTE 적용 후 라벨 분포:")
print(pd.Series(y_resampled).value_counts())

# 8. 데이터프레임으로 합치기
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['label'] = y_resampled

# 9. 증강된 데이터 CSV로 저장 (UTF-8)
df_resampled.to_csv('resampled_data.csv', encoding='utf-8', index=False)
print("\n증강 데이터 저장 완료: resampled_data.csv")


Saving summary.csv to summary (4).csv
데이터 불러오기 완료!
데이터 예시:
   전체시간(sec)  평균RPM  Stall증가시각_1  Stall증가시각_2  Stall증가위치_1  Stall증가위치_2  \
0      68.99   3.93       62.879       66.959      2095.90      2111.27   
1      62.81   5.06       46.719       56.877      1984.99      2368.42   
2      76.54   3.98       69.567       74.513      2272.41      2371.74   
3      80.75   4.64       74.818       78.751      2862.52      2902.45   
4      86.25   3.38       80.867       84.223      2242.74      2258.24   

   총회전거리(°)  최대RPM  RPM표준편차     label  
0   2111.67   7.93     1.86  abnormal  
1   2472.48  10.82     2.20  abnormal  
2   2371.92   7.73     1.42  abnormal  
3   2902.54   8.37     1.59  abnormal  
4   2259.05   7.16     1.14  abnormal  

라벨 분포:
label
abnormal    17
normal       3
Name: count, dtype: int64

SMOTE 적용 후 라벨 분포:
label
abnormal    17
normal      17
Name: count, dtype: int64

증강 데이터 저장 완료: resampled_data.csv


In [ ]:
import pandas as pd
import numpy as np

# 1. 원본 데이터 로드
try:
    df = pd.read_csv('resampled_data.csv')
except FileNotFoundError:
    print("Error: 'resampled_data.csv' 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

# 'label' 열을 제외한 수치형 열만 선택
numeric_cols = df.columns.drop('label')

# 각 범주별로 증강할 샘플 수 설정
augmentation_factors = {
    'normal': 1000,
    'abnormal': 1000
}

augmented_data_list = []

# 각 범주별로 데이터 증강
for label, factor in augmentation_factors.items():
    subset = df[df['label'] == label]

    # 원본 데이터는 항상 포함
    augmented_subset_list = [subset]

    # 증강할 데이터 수 계산
    num_to_augment = factor - subset.shape[0]

    if num_to_augment > 0:
        for _ in range(num_to_augment):
            original_sample = subset.sample(n=1, replace=True)

            # 노이즈 생성
            noise = np.random.normal(0, subset[numeric_cols].std() * 0.05, size=original_sample[numeric_cols].shape)

            # 원본 데이터에 노이즈를 더해 새로운 데이터 생성
            new_data = original_sample.copy()
            new_data[numeric_cols] += noise

            augmented_subset_list.append(new_data)

    augmented_subset = pd.concat(augmented_subset_list, ignore_index=True)
    augmented_data_list.append(augmented_subset)

# 모든 증강된 데이터를 하나로 합치기
final_df = pd.concat(augmented_data_list, ignore_index=True)

# --- 새로운 요구사항 반영 ---
# 1. 음수값이 있는 행 제거
# 모든 수치형 열에 대해 음수 값이 하나라도 있는 행을 제거
final_df = final_df[(final_df[numeric_cols] >= 0).all(axis=1)]

# 2. '최종회전수(rot)'와 'Stall최대값' 열 반올림
if 'Final_Rotation_rot' in final_df.columns:
    final_df['Final_Rotation_rot'] = final_df['Final_Rotation_rot'].round()
if 'Stall_Max_Value' in final_df.columns:
    final_df['Stall_Max_Value'] = final_df['Stall_Max_Value'].round()
# --- 수정 끝 ---


# 결과 확인
print("Original data shape:", df.shape)
print("Final data shape (after augmentation & filtering):", final_df.shape)
print("\nFinal data label distribution:\n", final_df['label'].value_counts())
print("\n최종 데이터프레임의 '최종회전수(rot)'와 'Stall최대값' 열:\n", final_df[['Final_Rotation_rot', 'Stall_Max_Value']].head())
print("\n최종 데이터프레임의 모든 값은 0 이상입니다:", (final_df[numeric_cols] >= 0).all().all())


# 최종 데이터프레임을 새로운 CSV 파일로 저장
final_df.to_csv('resampled_data_final_cleaned.csv', index=False)

Original data shape: (34, 10)
Final data shape (after augmentation & filtering): (1949, 10)

Final data label distribution:
 label
normal      1000
abnormal     949
Name: count, dtype: int64

최종 데이터프레임의 모든 값은 0 이상입니다: True
